# Genotyping VCF Analysis
This notebook details the analysis of VCFs of tumor sample genotyping data.

In [1]:
# first, import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from itables import show
from IPython.display import HTML, display
from warnings import simplefilter, filterwarnings
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
filterwarnings("ignore", category=UserWarning)
pd.set_option('display.max_columns', None)
%config InlineBackend.figure_format = 'retina'


In [9]:
%%bash
# head MN vcf
bcftools head -n 10 vcfs/SD0403/SD0403.normal.vcf.gz

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=AF,Number=A,Type=Float,Description="Allele fractions of alternate alleles in the tumor">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=F1R2,Number=R,Type=Integer,Description="Count of reads in F1R2 pair orientation supporting each allele">
##FORMAT=<ID=F2R1,Number=R,Type=Integer,Description="Count of reads in F2R1 pair orientation supporting each allele">
##FORMAT=<ID=FAD,Number=R,Type=Integer,Description="Count of fragments supporting each allele.">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PGT,Number=1,Type=String,Description="Physical phasing haplotype information, describing how the alternat

In [10]:
%%bash
# head TUM vcf
bcftools head -n 10 vcfs/SD0403/SD0403.tumour.vcf.gz

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FILTER=<ID=LowQual,Description="Low quality">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
##GATKCommandLine=<ID=HaplotypeCaller,CommandLine="HaplotypeCaller --dont-use-soft-clipped-bases true --output /home/ubuntu/repos/WES-SNV-discovery-workflow/data-holding-bay/SD0403.gatk4.all.germline.haps.vcf.gz --input /home/ubuntu/repos/WES-SNV-discovery-workflow/data-holding-bay/SD0403.sorted.RG-added.merged.dedup.recal.bam --reference refs/GRCh38.109.fa --tmp

In [ ]:
%%bash
cd vcfs/
for TUM_ID in SD0403 SD0507 SD0532 SD0560 SD0683 SD1225 SD0693 SD0781 SD1043 SD1182 SD1225; do
    cp ${TUM_ID}/*vcf.gz tmp/
    cp ${TUM_ID}/*.tbi tmp/
done;
bcftools merge tmp/*.vcf.gz -Oz -o merged.MN.TUM.vcf.gz;
vcftools --gzvcf merged.MN.TUM.vcf.gz --relatedness2 --out merged.MN.TUM;

if [ -f merged.MN.TUM.vcf.gz ]; then
	rm tmp/*
	rm merged.MN.TUM.vcf.gz
fi




VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf merged.MN.TUM.vcf.gz
	--out merged.MN.TUM
	--relatedness2

Using zlib version: 1.3.1
After filtering, kept 20 out of 20 Individuals
Outputting Individual Relatedness
	Relatedness: Only using biallelic sites.
After filtering, kept 6371192 out of a possible 6371192 Sites
Run Time = 32.00 seconds


### Plot the relatedness matrix

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Read the TSV file
df = pd.read_csv('/home/sufyazi/repos/WES-GENPIPIB/notebooks/vcfs/merged.MN.TUM.relatedness2', sep='\t')

# Create a pivot table
pivot_df = df.pivot(index='INDV1', columns='INDV2', values='RELATEDNESS_PHI')

# Fill missing values with their mirror values across the diagonal
for i in pivot_df.index:
    for j in pivot_df.columns:
        if pd.isna(pivot_df.loc[i,j]):
            pivot_df.loc[i,j] = pivot_df.loc[j,i]

# Create the heatmap
# Create a mask for the upper triangle
mask = np.triu(np.ones_like(pivot_df), k=1)

# Set figure size and DPI for higher resolution
plt.figure(figsize=(15, 12), dpi=300)

# Create the heatmap with mask
sns.heatmap(pivot_df, 
            cmap='Blues',
            annot=True,  # Show numbers in cells
            fmt='.3f',   # Format to 3 decimal places
            square=True, # Make cells square
            mask=mask,   # Apply the mask
            annot_kws={'size': 8},  # Adjust font size of numbers
            cbar_kws={'label': 'Relatedness Phi'})  # Add colorbar label

# Customize the plot
plt.title('Relatedness Phi Matrix', pad=20, size=14)
plt.xticks(rotation=45, ha='right', size=10)
plt.yticks(rotation=0, size=10)

# Adjust layout to prevent label cutoff
plt.tight_layout()
plt.show()

In [16]:
pivot_df

INDV2,SD0403_TUM,SD0507_TUM,SD0532_TUM,SD0560_TUM,SD0683_TUM,SD0693_TUM,SD0781_TUM,SD1043_MN,SD1043_TUM,SD1182_MN,SD1182_TUM,SD1225_MN,SD1225_TUM,SD403_MN,SD507_MN,SD532_MN,SD560_MN,SD683_MN,SD693_MN,SD781_MN
INDV1,,,,,,,,,,,,,,,,,,,,
SD0403_TUM,0.500000,0.057217,0.074804,0.074043,0.065398,0.056289,0.070302,0.045802,0.068309,0.047395,0.073016,0.042784,0.070788,0.071981,0.047249,0.042513,0.046871,0.041851,0.048978,0.044624
SD0507_TUM,0.057217,0.500000,0.074385,0.097956,0.060395,0.078538,0.055581,0.026226,0.103145,0.025362,0.063764,0.022651,0.063010,0.024341,0.047767,0.022191,0.027612,0.022890,0.029047,0.025336
SD0532_TUM,0.074804,0.074385,0.500000,0.095577,0.070964,0.069542,0.073804,0.040449,0.092886,0.040058,0.076627,0.036026,0.073166,0.038942,0.044394,0.056122,0.042065,0.036129,0.044197,0.040254
SD0560_TUM,0.074043,0.097956,0.095577,0.500000,0.077531,0.087319,0.071965,0.034518,0.130358,0.033402,0.078086,0.028958,0.077594,0.031806,0.040395,0.028710,0.060325,0.030344,0.037962,0.033351
SD0683_TUM,0.065398,0.060395,0.070964,0.077531,0.500000,0.058502,0.064362,0.040161,0.075235,0.039165,0.067161,0.035557,0.065014,0.037944,0.041589,0.035115,0.041415,0.060824,0.042437,0.038519
SD0693_TUM,0.056289,0.078538,0.069542,0.087319,0.058502,0.500000,0.054862,0.025491,0.091097,0.024790,0.059395,0.022054,0.057301,0.024279,0.029025,0.021856,0.026838,0.022739,0.044723,0.024950
SD0781_TUM,0.070302,0.055581,0.073804,0.071965,0.064362,0.054862,0.500000,0.044609,0.069238,0.044617,0.068792,0.041950,0.067586,0.043986,0.045498,0.041504,0.045867,0.040854,0.047125,0.067428
SD1043_MN,0.045802,0.026226,0.040449,0.034518,0.040161,0.025491,0.044609,0.500000,0.056618,0.087561,0.041790,0.089589,0.042464,0.088969,0.077316,0.087458,0.084146,0.082751,0.084084,0.083664
SD1043_TUM,0.068309,0.103145,0.092886,0.130358,0.075235,0.091097,0.069238,0.056618,0.500000,0.029797,0.073146,0.026210,0.074785,0.029082,0.037825,0.026140,0.033253,0.027922,0.035374,0.031278
